In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import scipy.sparse as scs
import torch
import torch.nn.functional as F

from recsys.utils import col
from recsys.metrics import ndcg_score, get_ndcg, get_apak
from recsys.models.nn import GMF

In [ ]:
device = "cuda"
torch.set_default_device(device)

### Last one

In [ ]:
X = pd.read_parquet("../data/ml-1m/split/X_last_one.parquet")
y = pd.read_parquet("../data/ml-1m/split/y_last_one.parquet")

In [ ]:
y_true = y[col.movie_code].to_numpy().reshape(-1, 1)

In [ ]:
user_movie_matrix = scs.coo_matrix(
    (X[col.rating], (X[col.user_code], X[col.movie_code]))
).tocsr()

user_movie_matrix[user_movie_matrix.nonzero()] = 1

In [ ]:
train_data = torch.utils.data.DataLoader(
    np.hstack(
        [
            np.arange(user_movie_matrix.shape[0]).reshape(-1, 1),
            user_movie_matrix.todense(),
        ]
    ),
    batch_size=32,
    shuffle=True,
    generator=torch.Generator(device=device),
)

#### GMF

In [ ]:
model = GMF(*user_movie_matrix.shape, 128).to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model
criterion = nn.BCEWithLogitsLoss()  # Choose your desired loss function
optimizer = optim.SGD(model.parameters(), lr=0.0001)
num_epochs = 50

# Define the gradient clipping value
max_norm = 1.0

# Training loop
train_losses = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_losses = 0

    for inputs in train_data:
        inputs = inputs.squeeze(1)
        uids, labels = inputs[:, 0], inputs[:, 1:]
        optimizer.zero_grad()
        outputs = model(uids)
        loss = criterion(outputs, labels.float())
        loss.backward()

        # Perform gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        optimizer.step()

        train_losses.append(loss.detach().item())
        running_losses += loss.detach().item()

    epoch_loss = running_losses / len(train_data)

    # Print the loss for each epoch
    # if epoch % 20 == 0 or epoch == num_epochs - 1:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [ ]:
scores = []
orders = []
with torch.no_grad():
    for inputs in train_data:
        inputs = inputs.squeeze(1)
        uids, labels = inputs[:, 0], inputs[:, 1:]
        orders.append(uids.cpu().numpy())
        scores.append(F.softmax(model(uids), dim=1).cpu().numpy())

retrieval =  np.vstack(scores)[np.argsort(np.hstack(orders))]
retrieval[user_movie_matrix.nonzero()] = -1
retrieval = np.argsort(retrieval, axis=1)[:, ::-1]

y_pred = retrieval[:, :10]

ndcg_score(y_true, y_pred)